# Page Fetching for Pagination Strategies

This notebook loads pagination plans from Stage 1 and executes the actual data fetching for each strategy.

## Strategies:
1. **Full Table** - Single query for entire table
2. **Row by Row** - Fetch each row individually using key values
3. **Attribute-based** - Fetch pages by partition values
4. **Classic Pagination** - Offset-based iterative fetching
5. **Range-based** - Fetch by defined ranges

## Output:
- CSV files: Aggregated table data (for metrics calculation)
- JSON files: Detailed execution logs with per-page metrics

## Setup and Imports

In [12]:
from __future__ import annotations

import json
import os
import time
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import openai

# ===== CONFIGURATION =====
# MAX_TABLES: Limit number of tables to process (None for all)
MAX_TABLES = 5

# MAX_PAGES: Limit pages per table per strategy (None for all, useful for testing)
MAX_PAGES = None

# PARALLEL_STRATEGIES: Run strategies in parallel per table
PARALLEL_STRATEGIES = True

# MAX_WORKERS: Number of parallel strategy executions per table
MAX_WORKERS = 5

# MAX_RETRIES: Number of retries for failed LLM calls
MAX_RETRIES = 3

# MAX_PAGINATION_PAGES: Failsafe for classic_pagination
MAX_PAGINATION_PAGES = 50

# Paths
ROOT = Path('.')
PROCESSING_ROOT = ROOT / 'processing'
STRATEGY_DIR = PROCESSING_ROOT / '1_strategy'
OUTPUT_ROOT = PROCESSING_ROOT / '2_fetching'

# Find the most recent strategy directory
strategy_subdirs = sorted([d for d in STRATEGY_DIR.iterdir() if d.is_dir()], reverse=True)
if not strategy_subdirs:
    raise FileNotFoundError(f"No strategy data found in {STRATEGY_DIR}")

LATEST_STRATEGY_DIR = strategy_subdirs[0]

# Find ground truth data directory
DATA_DIR = PROCESSING_ROOT / '0_data'
data_subdirs = sorted([d for d in DATA_DIR.iterdir() if d.is_dir()], reverse=True)
if not data_subdirs:
    raise FileNotFoundError(f"No ground truth data found in {DATA_DIR}")

LATEST_DATA_DIR = data_subdirs[0]

print('Strategy directory:', LATEST_STRATEGY_DIR.resolve())
print('Ground truth directory:', LATEST_DATA_DIR.resolve())
print('Output base:', OUTPUT_ROOT.resolve())

Strategy directory: /Users/bef/Desktop/TablePagination/processing/1_strategy/20251004_235228
Ground truth directory: /Users/bef/Desktop/TablePagination/processing/0_data/20251004_213355
Output base: /Users/bef/Desktop/TablePagination/processing/2_fetching


In [13]:
# Configure OpenRouter
# Read API key from file
api_key_file = ROOT / 'api_key.txt'
if not api_key_file.exists():
    raise ValueError('No API key found. Please create api_key.txt or set OPENROUTER_API_KEY environment variable')
with open(api_key_file, 'r') as f:
    OPENROUTER_API_KEY = f.read().strip()
print('✓ API key loaded from api_key.txt')

client = openai.OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

MODEL = 'x-ai/grok-code-fast-1'
print(f'Using model: {MODEL}')

✓ API key loaded from api_key.txt
Using model: x-ai/grok-code-fast-1


## Helper Functions

In [ ]:
def call_llm_with_metrics(prompt: str, retry_count: int = 0) -> Dict[str, Any]:
    """
    Call LLM and return response with detailed metrics.
    
    Returns dict with:
    - success: bool
    - response: str (if successful)
    - error: str (if failed)
    - metrics: dict with latency, tokens, prompt_length, response_length, etc.
    """
    start_time = time.time()
    prompt_length = len(prompt)
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        latency = time.time() - start_time
        content = response.choices[0].message.content
        
        # Check if content is None
        if content is None:
            return {
                'success': False,
                'error': 'Response content is None',
                'metrics': {
                    'latency': round(latency, 3),
                    'prompt_length': prompt_length,
                    'retry_count': retry_count,
                    'timestamp': datetime.now().isoformat()
                }
            }
        
        content = content.strip()
        response_length = len(content)
        
        # Extract token usage
        usage = response.usage
        prompt_tokens = usage.prompt_tokens if usage else 0
        completion_tokens = usage.completion_tokens if usage else 0
        total_tokens = usage.total_tokens if usage else 0
        
        return {
            'success': True,
            'response': content,
            'metrics': {
                'latency': round(latency, 3),
                'prompt_tokens': prompt_tokens,
                'completion_tokens': completion_tokens,
                'total_tokens': total_tokens,
                'prompt_length': prompt_length,
                'response_length': response_length,
                'retry_count': retry_count,
                'timestamp': datetime.now().isoformat()
            }
        }
    except Exception as e:
        latency = time.time() - start_time
        return {
            'success': False,
            'error': str(e),
            'metrics': {
                'latency': round(latency, 3),
                'prompt_length': prompt_length,
                'retry_count': retry_count,
                'timestamp': datetime.now().isoformat()
            }
        }


def call_llm_with_retries(prompt: str, max_retries: int = MAX_RETRIES) -> Dict[str, Any]:
    """Call LLM with retry logic."""
    for attempt in range(max_retries):
        result = call_llm_with_metrics(prompt, retry_count=attempt)
        if result['success']:
            return result
        print(f"  Retry {attempt + 1}/{max_retries} after error: {result['error']}")
        time.sleep(1)  # Brief delay between retries
    
    return result  # Return last failed attempt


def parse_json_from_response(response: str) -> Tuple[Optional[List[Dict]], Dict[str, Any]]:
    """
    Extract and parse JSON from LLM response.
    
    Returns:
    - parsed_data: List of dicts or None
    - parse_metrics: dict with parse_success, json_start_pos, json_end_pos
    """
    metrics = {
        'parse_success': False,
        'json_start_pos': -1,
        'json_end_pos': -1
    }
    
    try:
        # Find JSON array in response
        start_pos = response.find('[')
        end_pos = response.rfind(']')
        
        if start_pos == -1 or end_pos == -1:
            return None, metrics
        
        json_str = response[start_pos:end_pos + 1]
        metrics['json_start_pos'] = start_pos
        metrics['json_end_pos'] = end_pos + 1
        
        parsed = json.loads(json_str)
        
        if isinstance(parsed, list):
            metrics['parse_success'] = True
            return parsed, metrics
        
        return None, metrics
        
    except json.JSONDecodeError as e:
        metrics['parse_error'] = str(e)
        return None, metrics

## Strategy 1: Full Table

In [15]:
def fetch_full_table(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetch entire table in one query with example row.
    """
    meta = plan['metadata']
    table_title = meta['table_title']
    columns = meta['columns']
    table_id = plan['table_id']
    
    # Load ground truth to get example row
    gt_file = LATEST_DATA_DIR / f"{table_id}.csv"
    df_ref = pd.read_csv(gt_file)
    example_row = df_ref.iloc[0]
    
    # Format example in JSON
    example_json = ', '.join([f'"{col}": "{example_row[col]}"' for col in columns])
    
    prompt = f"""You are a retriever of facts.
We want to create a table with detailed information about {table_title}.
Return all rows from the table.

RESPONSE FORMAT (JSON array):
[{{
    {example_json}
}}]"""
    
    print(f"  Fetching full table...")
    
    result = call_llm_with_retries(prompt)
    
    page_data = {
        'page_number': 1,
        'prompt': prompt,
        'raw_response': result.get('response', ''),
        'error': result.get('error'),
        **result['metrics']
    }
    
    if result['success']:
        parsed_data, parse_metrics = parse_json_from_response(result['response'])
        page_data.update(parse_metrics)
        page_data['rows_returned'] = len(parsed_data) if parsed_data else 0
        page_data['parsed_data'] = parsed_data
    else:
        page_data['parse_success'] = False
        page_data['rows_returned'] = 0
        page_data['parsed_data'] = []
    
    return {
        'pages': [page_data],
        'all_rows': page_data['parsed_data']
    }
    

## Strategy 2: Row by Row

In [16]:
def fetch_row_by_row(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetch each row individually using key values from plan with example.
    """
    meta = plan['metadata']
    config = plan['pagination_config']
    
    table_title = meta['table_title']
    columns = meta['columns']
    key_columns = config['key_columns']
    key_values = config['key_values']
    table_id = plan['table_id']
    
    # Load ground truth to get example row
    gt_file = LATEST_DATA_DIR / f"{table_id}.csv"
    df_ref = pd.read_csv(gt_file)
    example_row = df_ref.iloc[0]
    
    pages = []
    all_rows = []
    
    # Apply MAX_PAGES limit if set
    keys_to_fetch = key_values[:MAX_PAGES] if MAX_PAGES else key_values
    total_keys = len(key_values)
    
    if MAX_PAGES and len(key_values) > MAX_PAGES:
        print(f"  Limited to first {MAX_PAGES} rows out of {total_keys}")
    
    for page_num, key_combo in enumerate(keys_to_fetch, 1):
        # Build WHERE clause for this row
        where_clauses = [f"{k}={key_combo.get(k.lower().replace(' ', '_'), 'UNKNOWN')}" 
                        for k in key_columns]
        where_str = " and ".join(where_clauses)
        
        # Format example with current key values
        example_json_parts = []
        for col in columns:
            col_norm = col.lower().replace(' ', '_')
            if col_norm in key_combo:
                example_json_parts.append(f'"{col}": "{key_combo[col_norm]}"')
            else:
                example_json_parts.append(f'"{col}": "{example_row[col]}"')
        example_json = ', '.join(example_json_parts)
        
        prompt = f"""You are a retriever of facts.
We want to retrieve a specific row from a table about {table_title}.
Return the row where {where_str}.

RESPONSE FORMAT (JSON array with single object):
[{{
    {example_json}
}}]"""
        
        print(f"  Fetching row {page_num}/{len(keys_to_fetch)}...")
        
        result = call_llm_with_retries(prompt)
        
        page_data = {
            'page_number': page_num,
            'key_values': key_combo,
            'prompt': prompt,
            'raw_response': result.get('response', ''),
            'error': result.get('error'),
            **result['metrics']
        }
        
        if result['success']:
            parsed_data, parse_metrics = parse_json_from_response(result['response'])
            page_data.update(parse_metrics)
            page_data['rows_returned'] = len(parsed_data) if parsed_data else 0
            page_data['parsed_data'] = parsed_data
            
            if parsed_data:
                all_rows.extend(parsed_data)
        else:
            page_data['parse_success'] = False
            page_data['rows_returned'] = 0
            page_data['parsed_data'] = []
        
        pages.append(page_data)
    
    return {
        'pages': pages,
        'all_rows': all_rows
    }

## Strategy 3: Attribute-based

In [17]:
def fetch_attribute_based(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetch pages by partition values with example.
    """
    meta = plan['metadata']
    config = plan['pagination_config']
    
    table_title = meta['table_title']
    columns = meta['columns']
    partition_column = config['partition_column']
    partition_values = config['partition_values']
    table_id = plan['table_id']
    
    # Load ground truth to get example row
    gt_file = LATEST_DATA_DIR / f"{table_id}.csv"
    df_ref = pd.read_csv(gt_file)
    example_row = df_ref.iloc[0]
    
    pages = []
    all_rows = []
    
    # Apply MAX_PAGES limit
    values_to_fetch = partition_values[:MAX_PAGES] if MAX_PAGES else partition_values
    total_values = len(partition_values)
    
    if MAX_PAGES and len(partition_values) > MAX_PAGES:
        print(f"  Limited to first {MAX_PAGES} partitions out of {total_values}")
    
    for page_num, value in enumerate(values_to_fetch, 1):
        # Format example
        example_json = ', '.join([f'"{col}": "{example_row[col]}"' for col in columns])
        
        prompt = f"""You are a retriever of facts.
We want to retrieve rows from a table about {table_title}.
Return all rows where {partition_column} = {value}.

RESPONSE FORMAT (JSON array):
[{{
    {example_json}
}}]"""
        
        print(f"  Fetching partition {page_num}/{len(values_to_fetch)}: {partition_column}={value}...")
        
        result = call_llm_with_retries(prompt)
        
        page_data = {
            'page_number': page_num,
            'partition_value': value,
            'prompt': prompt,
            'raw_response': result.get('response', ''),
            'error': result.get('error'),
            **result['metrics']
        }
        
        if result['success']:
            parsed_data, parse_metrics = parse_json_from_response(result['response'])
            page_data.update(parse_metrics)
            page_data['rows_returned'] = len(parsed_data) if parsed_data else 0
            page_data['parsed_data'] = parsed_data
            
            if parsed_data:
                all_rows.extend(parsed_data)
        else:
            page_data['parse_success'] = False
            page_data['rows_returned'] = 0
            page_data['parsed_data'] = []
        
        pages.append(page_data)
    
    return {
        'pages': pages,
        'all_rows': all_rows
    }

## Strategy 4: Classic Pagination

In [18]:
def fetch_classic_pagination(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    Iterative offset-based pagination with example.
    Stops when LLM says "no more rows" or hits MAX_PAGINATION_PAGES failsafe.
    """
    meta = plan['metadata']
    config = plan['pagination_config']
    
    table_title = meta['table_title']
    columns = meta['columns']
    page_size = config['page_size']
    sort_order = config['sort_order']
    table_id = plan['table_id']
    
    # Load ground truth to get example row
    gt_file = LATEST_DATA_DIR / f"{table_id}.csv"
    df_ref = pd.read_csv(gt_file)
    example_row = df_ref.iloc[0]
    
    pages = []
    all_rows = []
    offset = 0
    page_num = 0
    
    max_pages = min(MAX_PAGES, MAX_PAGINATION_PAGES) if MAX_PAGES else MAX_PAGINATION_PAGES
    
    while page_num < max_pages:
        page_num += 1
        
        # Format example
        example_json = ', '.join([f'"{col}": "{example_row[col]}"' for col in columns])
        
        prompt = f"""You are a retriever of facts.
We want to retrieve page {page_num} from a table about {table_title}.
Return rows {offset + 1} to {offset + page_size}, sorted by {', '.join(sort_order)}.

If there are no more rows at this offset, respond with exactly: "no more rows"

RESPONSE FORMAT (JSON array):
[{{
    {example_json}
}}]"""
        
        print(f"  Fetching page {page_num} (offset {offset}, size {page_size})...")
        
        result = call_llm_with_retries(prompt)
        
        page_data = {
            'page_number': page_num,
            'offset': offset,
            'page_size': page_size,
            'prompt': prompt,
            'raw_response': result.get('response', ''),
            'error': result.get('error'),
            **result['metrics']
        }
        
        # Check for stop condition
        if result['success']:
            response_lower = result['response'].lower()
            if 'no more rows' in response_lower and '[' not in result['response']:
                print(f"  Stopping: LLM indicated no more rows")
                page_data['parse_success'] = True
                page_data['rows_returned'] = 0
                page_data['parsed_data'] = []
                page_data['stop_reason'] = 'no_more_rows'
                pages.append(page_data)
                break
            
            parsed_data, parse_metrics = parse_json_from_response(result['response'])
            page_data.update(parse_metrics)
            page_data['rows_returned'] = len(parsed_data) if parsed_data else 0
            page_data['parsed_data'] = parsed_data
            
            if parsed_data:
                all_rows.extend(parsed_data)
                offset += page_size
            else:
                # Empty result, assume end
                print(f"  Stopping: Empty result")
                page_data['stop_reason'] = 'empty_result'
                pages.append(page_data)
                break
        else:
            page_data['parse_success'] = False
            page_data['rows_returned'] = 0
            page_data['parsed_data'] = []
        
        pages.append(page_data)
    
    if page_num >= max_pages:
        print(f"  Stopping: Hit max pages limit ({max_pages})")
    
    return {
        'pages': pages,
        'all_rows': all_rows
    }

## Strategy 5: Range-based

In [19]:
def fetch_range_based(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetch pages by defined ranges with example.
    """
    meta = plan['metadata']
    config = plan['pagination_config']
    
    table_title = meta['table_title']
    columns = meta['columns']
    partition_column = config['partition_column']
    ranges = config['ranges']
    table_id = plan['table_id']
    
    # Load ground truth to get example row
    gt_file = LATEST_DATA_DIR / f"{table_id}.csv"
    df_ref = pd.read_csv(gt_file)
    example_row = df_ref.iloc[0]
    
    pages = []
    all_rows = []
    
    # Apply MAX_PAGES limit
    ranges_to_fetch = ranges[:MAX_PAGES] if MAX_PAGES else ranges
    total_ranges = len(ranges)
    
    if MAX_PAGES and len(ranges) > MAX_PAGES:
        print(f"  Limited to first {MAX_PAGES} ranges out of {total_ranges}")
    
    for page_num, range_spec in enumerate(ranges_to_fetch, 1):
        gte = range_spec.get('gte', '')
        lt = range_spec.get('lt', '')
        
        # Format example
        example_json = ', '.join([f'"{col}": "{example_row[col]}"' for col in columns])
        
        prompt = f"""You are a retriever of facts.
We want to retrieve rows from a table about {table_title}.
Return all rows where {partition_column} >= {gte} and {partition_column} < {lt}.

RESPONSE FORMAT (JSON array):
[{{
    {example_json}
}}]"""
        
        print(f"  Fetching range {page_num}/{len(ranges_to_fetch)}: {partition_column} [{gte}, {lt})...")
        
        result = call_llm_with_retries(prompt)
        
        page_data = {
            'page_number': page_num,
            'range': range_spec,
            'prompt': prompt,
            'raw_response': result.get('response', ''),
            'error': result.get('error'),
            **result['metrics']
        }
        
        if result['success']:
            parsed_data, parse_metrics = parse_json_from_response(result['response'])
            page_data.update(parse_metrics)
            page_data['rows_returned'] = len(parsed_data) if parsed_data else 0
            page_data['parsed_data'] = parsed_data
            
            if parsed_data:
                all_rows.extend(parsed_data)
        else:
            page_data['parse_success'] = False
            page_data['rows_returned'] = 0
            page_data['parsed_data'] = []
        
        pages.append(page_data)
    
    return {
        'pages': pages,
        'all_rows': all_rows
    }

## Main Execution: Process All Strategies

In [20]:
# Strategy function mapping
STRATEGY_FUNCTIONS = {
    'full_table': fetch_full_table,
    'row_by_row': fetch_row_by_row,
    'attribute_based': fetch_attribute_based,
    'classic_pagination': fetch_classic_pagination,
    'range_based': fetch_range_based
}

# Load all strategies
strategies_to_process = []
for strategy_name in STRATEGY_FUNCTIONS.keys():
    strategy_dir = LATEST_STRATEGY_DIR / strategy_name
    if not strategy_dir.exists():
        print(f"Skipping {strategy_name}: directory not found")
        continue
    
    json_files = sorted(strategy_dir.glob('*.json'))
    # Filter out error files
    json_files = [f for f in json_files if not f.name.startswith('_')]
    
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            plan = json.load(f)
        strategies_to_process.append((strategy_name, plan))

print(f"Found {len(strategies_to_process)} strategy-table combinations")

# Filter to only tables that have plans for ALL strategies (apples-to-apples comparison)
from collections import Counter
all_strategies = list(STRATEGY_FUNCTIONS.keys())
table_strategy_counts = Counter(plan['table_id'] for _, plan in strategies_to_process)
tables_with_all_strategies = [table_id for table_id, count in table_strategy_counts.items() 
                               if count == len(all_strategies)]

print(f"\nFiltering to tables with plans for all {len(all_strategies)} strategies:")
print(f"  Total unique tables: {len(table_strategy_counts)}")
print(f"  Tables with all strategies: {len(tables_with_all_strategies)}")

strategies_to_process = [(s, p) for s, p in strategies_to_process 
                         if p['table_id'] in tables_with_all_strategies]
print(f"  Filtered to {len(strategies_to_process)} strategy-table combinations")

# Apply MAX_TABLES limit across all strategies
if MAX_TABLES:
    # Group by table_id to ensure we process complete sets
    table_ids = list(set(p['table_id'] for _, p in strategies_to_process))[:MAX_TABLES]
    strategies_to_process = [(s, p) for s, p in strategies_to_process if p['table_id'] in table_ids]
    print(f"Limited to {len(strategies_to_process)} combinations for {len(table_ids)} tables")

print(f"Processing {len(strategies_to_process)} strategy-table combinations...")

Found 48 strategy-table combinations

Filtering to tables with plans for all 1 strategies:
  Total unique tables: 48
  Tables with all strategies: 48
  Filtered to 48 strategy-table combinations
Limited to 5 combinations for 5 tables
Processing 5 strategy-table combinations...


In [21]:
# Create output directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = OUTPUT_ROOT / timestamp
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {output_dir}")

# Create subdirectories for each strategy
for strategy_name in STRATEGY_FUNCTIONS.keys():
    strategy_dir = output_dir / strategy_name
    strategy_dir.mkdir(exist_ok=True)

# Group strategies by table_id for parallel execution
from collections import defaultdict
tables_with_strategies = defaultdict(list)
for strategy_name, plan in strategies_to_process:
    table_id = plan['table_id']
    tables_with_strategies[table_id].append((strategy_name, plan))

print(f"\nGrouped into {len(tables_with_strategies)} tables with strategies")

# Process each table with parallel strategy execution
results_summary = []


def process_strategy(strategy_name: str, plan: Dict[str, Any]) -> Dict[str, Any]:
    """Process a single strategy-table combination."""
    table_id = plan['table_id']
    table_name = plan['table_name']
    
    try:
        # Execute fetching strategy
        fetch_func = STRATEGY_FUNCTIONS[strategy_name]
        fetch_result = fetch_func(plan)
        
        pages = fetch_result['pages']
        all_rows = fetch_result['all_rows']
        
        # Calculate aggregated metrics
        total_pages = len(pages)
        successful_pages = sum(1 for p in pages if p.get('parse_success', False))
        failed_pages = total_pages - successful_pages
        
        total_latency = sum(p.get('latency', 0) for p in pages)
        avg_latency = total_latency / total_pages if total_pages > 0 else 0
        
        total_tokens = sum(p.get('total_tokens', 0) for p in pages)
        total_llm_calls = sum(1 + p.get('retry_count', 0) for p in pages)
        
        total_rows_fetched = len(all_rows)
        
        # Check for duplicate rows
        unique_rows = set()
        duplicate_count = 0
        for row in all_rows:
            row_key = tuple(sorted(row.items()))
            if row_key in unique_rows:
                duplicate_count += 1
            unique_rows.add(row_key)
        
        # Check column consistency
        if all_rows:
            column_sets = [set(row.keys()) for row in all_rows]
            columns_consistent = all(cs == column_sets[0] for cs in column_sets)
        else:
            columns_consistent = True
        
        error_rate = failed_pages / total_pages if total_pages > 0 else 0
        
        # Build execution summary
        execution_summary = {
            'table_id': table_id,
            'table_name': table_name,
            'strategy': strategy_name,
            'metadata': plan['metadata'],
            'pagination_config': plan['pagination_config'],
            'execution_metadata': {
                'timestamp': timestamp,
                'total_pages': total_pages,
                'successful_pages': successful_pages,
                'failed_pages': failed_pages,
                'total_llm_calls': total_llm_calls,
                'total_latency': round(total_latency, 3),
                'avg_latency': round(avg_latency, 3),
                'total_tokens': total_tokens,
                'total_rows_fetched': total_rows_fetched,
                'unique_rows': len(unique_rows),
                'duplicate_rows': duplicate_count,
                'columns_consistent': columns_consistent,
                'error_rate': round(error_rate, 4)
            },
            'pages': pages
        }
        
        # Save JSON log
        json_path = output_dir / strategy_name / f"{table_id}.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(execution_summary, f, indent=2, ensure_ascii=False)
        
        # Save CSV if we have data
        if all_rows:
            try:
                df = pd.DataFrame(all_rows)
                csv_path = output_dir / strategy_name / f"{table_id}.csv"
                df.to_csv(csv_path, index=False, encoding='utf-8')
                print(f"  ✓ {strategy_name}/{table_id}: {len(df)} rows, {total_tokens} tokens, {total_latency:.2f}s")
            except Exception as e:
                print(f"  ✗ {strategy_name}/{table_id}: CSV save failed: {e}")
        else:
            print(f"  ⚠ {strategy_name}/{table_id}: No rows to save")
        
        return {
            'strategy': strategy_name,
            'table_id': table_id,
            'success': True,
            **execution_summary['execution_metadata']
        }
        
    except Exception as e:
        print(f"  ✗ {strategy_name}/{table_id}: Error: {e}")
        return {
            'strategy': strategy_name,
            'table_id': table_id,
            'success': False,
            'error': str(e)
        }


# Process each table with parallel strategies
for table_num, (table_id, strategies) in enumerate(tables_with_strategies.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{table_num}/{len(tables_with_strategies)}] Processing table: {table_id}")
    print(f"{'='*70}")
    print(f"Strategies: {', '.join(s for s, _ in strategies)}")
    
    if PARALLEL_STRATEGIES:
        # Execute strategies in parallel
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = {
                executor.submit(process_strategy, strategy_name, plan): (strategy_name, plan['table_id'])
                for strategy_name, plan in strategies
            }
            
            for future in as_completed(futures):
                result = future.result()
                results_summary.append(result)
    else:
        # Sequential execution (for debugging)
        for strategy_name, plan in strategies:
            print(f"\n  {strategy_name.upper()}")
            result = process_strategy(strategy_name, plan)
            results_summary.append(result)

print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
print(f"Processed: {len(results_summary)} strategy-table combinations")
print(f"Successful: {sum(1 for r in results_summary if r['success'])}")
print(f"Failed: {sum(1 for r in results_summary if not r['success'])}")
print(f"{'='*70}")

Output directory: processing/2_fetching/20251005_002153

Grouped into 5 tables with strategies

[1/5] Processing table: 16_curling_teams_women_2013_2014
Strategies: full_table
  Fetching full table...
  ✓ full_table/16_curling_teams_women_2013_2014: 21 rows, 2945 tokens, 13.07s

[2/5] Processing table: 20_new_zealand_demographics_1921_2011
Strategies: full_table
  Fetching full table...
  ✓ full_table/16_curling_teams_women_2013_2014: 21 rows, 2945 tokens, 13.07s

[2/5] Processing table: 20_new_zealand_demographics_1921_2011
Strategies: full_table
  Fetching full table...
  ✗ full_table/20_new_zealand_demographics_1921_2011: Error: object of type 'NoneType' has no len()

[3/5] Processing table: 36_south_cambridgeshire_district_council_1973_2012
Strategies: full_table
  Fetching full table...
  ✗ full_table/20_new_zealand_demographics_1921_2011: Error: object of type 'NoneType' has no len()

[3/5] Processing table: 36_south_cambridgeshire_district_council_1973_2012
Strategies: full_tabl

## Save Final Summary

In [22]:
# Calculate aggregate statistics across all strategies
execution_end_time = datetime.now()
# Parse timestamp: format is YYYYMMDD_HHMMSS
execution_start_time = datetime.strptime(timestamp, '%Y%m%d_%H%M%S')

total_execution_time = (execution_end_time - execution_start_time).total_seconds()

# Aggregate metrics
total_tables_processed = len(set(r['table_id'] for r in results_summary if r['success']))
total_strategies_run = len([r for r in results_summary if r['success']])
total_failures = len([r for r in results_summary if not r['success']])

successful_results = [r for r in results_summary if r.get('success', False)]

total_pages_fetched = sum(r.get('total_pages', 0) for r in successful_results)
total_llm_calls_made = sum(r.get('total_llm_calls', 0) for r in successful_results)
total_tokens_used = sum(r.get('total_tokens', 0) for r in successful_results)
total_latency_seconds = sum(r.get('total_latency', 0) for r in successful_results)
total_rows_fetched = sum(r.get('total_rows_fetched', 0) for r in successful_results)

# Calculate averages
avg_latency_per_page = total_latency_seconds / total_pages_fetched if total_pages_fetched > 0 else 0
avg_tokens_per_call = total_tokens_used / total_llm_calls_made if total_llm_calls_made > 0 else 0
avg_pages_per_strategy = total_pages_fetched / total_strategies_run if total_strategies_run > 0 else 0

# Collect all errors
errors_list = [
    {
        'strategy': r['strategy'],
        'table_id': r['table_id'],
        'error': r.get('error', 'Unknown error')
    }
    for r in results_summary if not r.get('success', False)
]

# Strategy breakdown
strategy_breakdown = {}
for strategy in STRATEGY_FUNCTIONS.keys():
    strategy_results = [r for r in successful_results if r.get('strategy') == strategy]
    if strategy_results:
        strategy_breakdown[strategy] = {
            'tables_processed': len(strategy_results),
            'total_pages': sum(r.get('total_pages', 0) for r in strategy_results),
            'total_tokens': sum(r.get('total_tokens', 0) for r in strategy_results),
            'total_latency': round(sum(r.get('total_latency', 0) for r in strategy_results), 3),
            'avg_latency': round(sum(r.get('avg_latency', 0) for r in strategy_results) / len(strategy_results), 3),
            'total_rows': sum(r.get('total_rows_fetched', 0) for r in strategy_results),
            'error_rate': round(sum(r.get('error_rate', 0) for r in strategy_results) / len(strategy_results), 4)
        }

# Build comprehensive summary
summary = {
    'timestamp': timestamp,
    'execution_time': {
        'start': timestamp,
        'end': execution_end_time.strftime('%Y%m%d_%H%M%S'),
        'total_seconds': round(total_execution_time, 2),
        'total_minutes': round(total_execution_time / 60, 2)
    },
    'configuration': {
        'strategy_directory': str(LATEST_STRATEGY_DIR),
        'max_tables': MAX_TABLES,
        'max_pages': MAX_PAGES,
        'max_retries': MAX_RETRIES,
        'max_pagination_pages': MAX_PAGINATION_PAGES,
        'model': MODEL
    },
    'aggregate_metrics': {
        'total_tables_processed': total_tables_processed,
        'total_strategy_table_combinations': total_strategies_run,
        'total_failures': total_failures,
        'success_rate': round(total_strategies_run / (total_strategies_run + total_failures), 4) if (total_strategies_run + total_failures) > 0 else 0,
        'total_pages_fetched': total_pages_fetched,
        'total_llm_calls': total_llm_calls_made,
        'total_tokens_used': total_tokens_used,
        'total_latency_seconds': round(total_latency_seconds, 2),
        'total_rows_fetched': total_rows_fetched,
        'avg_latency_per_page': round(avg_latency_per_page, 3),
        'avg_tokens_per_call': round(avg_tokens_per_call, 1),
        'avg_pages_per_strategy': round(avg_pages_per_strategy, 1)
    },
    'strategy_breakdown': strategy_breakdown,
    'errors': errors_list,
    'detailed_results': results_summary
}

summary_file = output_dir / '_summary.json'
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\n{'='*70}")
print("EXECUTION SUMMARY")
print(f"{'='*70}")
print(f"Total execution time: {summary['execution_time']['total_minutes']:.2f} minutes")
print(f"Tables processed: {total_tables_processed}")
print(f"Strategy combinations: {total_strategies_run} successful, {total_failures} failed")
print(f"Total pages fetched: {total_pages_fetched}")
print(f"Total LLM calls: {total_llm_calls_made}")
print(f"Total tokens used: {total_tokens_used:,}")
print(f"Total rows fetched: {total_rows_fetched}")
print(f"Avg latency per page: {avg_latency_per_page:.2f}s")
print(f"Avg tokens per call: {avg_tokens_per_call:.1f}")
if errors_list:
    print(f"\n⚠ {len(errors_list)} errors occurred (see _summary.json for details)")
print(f"\nFinal summary saved to {summary_file}")
print(f"All results saved to {output_dir}")
print(f"{'='*70}")


EXECUTION SUMMARY
Total execution time: 2.33 minutes
Tables processed: 4
Strategy combinations: 4 successful, 1 failed
Total pages fetched: 4
Total LLM calls: 4
Total tokens used: 16,064
Total rows fetched: 85
Avg latency per page: 22.83s
Avg tokens per call: 4016.0

⚠ 1 errors occurred (see _summary.json for details)

Final summary saved to processing/2_fetching/20251005_002153/_summary.json
All results saved to processing/2_fetching/20251005_002153


## Next Steps

The fetched data has been saved to `processing/2_fetching/<timestamp>/`.

Each strategy subdirectory contains:
- **JSON files**: Detailed execution logs with per-page metrics, prompts, responses, and aggregated statistics
- **CSV files**: Aggregated table data ready for metrics calculation

### Metrics Available:

**Per-page metrics:**
- latency, tokens (prompt/completion/total), retry_count
- parse_success, rows_returned, JSON extraction position
- timestamp, raw_response, parsed_data

**Per-table-strategy metrics:**
- total_pages, successful/failed_pages, total_llm_calls
- total/avg latency, total_tokens
- total_rows_fetched, unique_rows, duplicate_rows
- columns_consistent, error_rate

### To calculate accuracy metrics:
Use the CSV files with the evaluation logic from `X101_Calculate_Metrics.ipynb` to compute:
- Keys F1, Precision, Recall
- Non-keys F1, Precision, Recall
- Overall F1, Precision, Recall